In [2]:
import os
import openai
from openai import OpenAI
from dotenv import load_dotenv
import hashlib
import json
import time

In [2]:
load_dotenv()
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")

In [3]:
client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")

cache = {}  # Global cache

def display_cache_history(cache):
    """
    Display the conversation history stored in the cache.
    
    Parameters:
        cache (dict): The cache containing conversation history.
    """
    for cache_key, response in cache.items():
        print(f"Cache Key: {cache_key}")
        print("Messages:")
        
        # Extract messages from the nested response structure
        if hasattr(response, 'choices'):
            for choice in response.choices:
                if hasattr(choice, 'message') and hasattr(choice.message, 'content'):
                    print(f"  {choice.message.role.capitalize()}: {choice.message.content}")
        
        print("-" * 40)  # Separator for readability

def DeepseekV3(user_message, messages=None, reset_conversation=False, show_cache=False):
    """
    Function to interact with the DeepSeek API in a conversational manner with caching.
    
    Parameters:
        user_message (str): The message to send to the assistant.
        messages (list, optional): The conversation history. Defaults to None.
        reset_conversation (bool, optional): If True, starts a new conversation. Defaults to False.
        show_cache (bool, optional): If True, displays the cache history before the response. Defaults to False.
        
    Returns:
        list: The updated conversation history including the assistant's response.
    """
    if reset_conversation or messages is None:
        # Start a new conversation with a system message
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": user_message}
        ]
    else:
        # Append the user's message to the existing conversation
        messages.append({"role": "user", "content": user_message})
    
    # Create a unique key for the cache
    cache_key = hashlib.md5(json.dumps(messages, sort_keys=True).encode()).hexdigest()
    
    # Check if the response is in the cache
    if cache_key in cache:
        print("Using cached response.")
        response = cache[cache_key]
    else:
        print("Making API call.")
        # Send the API request
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=messages
        )
        # Store the response in the cache
        cache[cache_key] = response
    
    # Display the cache history if requested
    if show_cache:
        print("\nCache History:")
        display_cache_history(cache)
    
    # Get the assistant's response content
    assistant_response = response.choices[0].message.content
    
    # Append the assistant's response to the conversation history
    messages.append({"role": "assistant", "content": assistant_response})
    
    # Print the assistant's response
    print(f"\nAssistant Response:\n{assistant_response}")
    
    return messages

In [4]:
# Start a new conversation and display the cache
messages = DeepseekV3("What did Mao Zedong do wrong", reset_conversation=True, show_cache=True)

Making API call.

Cache History:
Cache Key: 063e44b8951c6263e81ac002d0a669d8
Messages:
  Assistant: Mao Zedong, the founding father of the People's Republic of China, is a highly controversial figure in history. While he is celebrated for his role in unifying China and ending decades of foreign domination and civil war, his leadership is also associated with significant failures and atrocities. Here are some of the most widely criticized aspects of Mao's rule:

### 1. **The Great Leap Forward (1958–1962)**
   - **Policy Failures**: Mao's campaign to rapidly industrialize China and collectivize agriculture led to economic disaster. Poor planning, unrealistic targets, and forced implementation caused widespread famine.
   - **Human Cost**: An estimated **20 to 45 million people** died due to starvation, disease, and overwork during this period, making it one of the deadliest man-made disasters in history.

### 2. **The Cultural Revolution (1966–1976)**
   - **Social Chaos**: Mao launched

In [15]:
# Continue the conversation without displaying the cache
messages = DeepseekV3("Call me the unemp", messages=messages)

Making API call.

Assistant Response:
It’s tough out there, but don’t lose hope! Here are some actionable steps to navigate the oversaturated job market and build your career, even without experience:

---

### 1. **Expand Your Skill Set**
   - **Learn In-Demand Tools**: If you’re in Data Science or Data Engineering, focus on tools like Snowflake, Databricks, Apache Airflow, or cloud platforms (AWS, GCP, Azure).  
   - **Add SWE Skills**: Learn Python, Java, or Go for backend development, or pick up frontend skills (React, JavaScript) to diversify your profile.  
   - **Certifications**: Get certified in cloud platforms, data engineering, or machine learning (e.g., AWS Certified Data Analytics, Google Cloud Data Engineer).

---

### 2. **Build a Portfolio**
   - **Personal Projects**: Work on real-world projects (e.g., building a data pipeline, creating a dashboard, or deploying a machine learning model).  
   - **Open Source Contributions**: Contribute to open-source projects on GitHu

In [14]:
# Start another new conversation and display the cache
messages = DeepseekV3("Make a dark joke about unemployed Data Science grads or data engineer grad", reset_conversation=True, show_cache=False)

Making API call.

Assistant Response:
Sure, here's a dark joke for you:

"Why did the unemployed Data Science grad bring a ladder to the job interview?  
Because they heard the bar for entry-level positions was set *way too high*... and they’re still climbing to reach it."

(Too real? 😅)


In [3]:
load_dotenv()
CODESTRAL_MAMBA_API_KEY = os.getenv("CODESTRAL_MAMBA_API_KEY")

In [9]:
import os
from mistralai import Mistral, UserMessage

# Replace with your actual API key
model = "codestral-mamba-latest"

client = Mistral(api_key=CODESTRAL_MAMBA_API_KEY)

def chat_mistral(prompt: str):
    messages = [
        UserMessage(content=prompt)
    ]

    # No streaming
    chat_response = client.chat.complete(
        model=model,
        messages=messages,
    )

    return chat_response.choices[0].message.content

# Example usage
prompt = """tạo 5 câu hỏi từ bộ luật này và dùng tiếng việt. DO NOT WRITE IN ENGLISH ONLY VIETNAMESE;
        "law_title": "Luật Giáo Dục",
        "law_number": "Luật số: 43/2019/QH14",
        "chapter_title": "Chương I",
        "article_number": "Điều 2",
        "title": "Mục tiêu giáo dục",
        "content": "Mục tiêu giáo dục nhằm phát triển toàn diện con người Việt Nam có đạo đức, tri thức, văn hóa, sức khỏe, thẩm mỹ và nghề nghiệp; có phẩm chất, năng lực và ý thức công dân; có lòng yêu nước, tinh thần dân tộc, trung thành với lý tưởng độc lập dân tộc và chủ nghĩa xã hội; phát huy tiềm năng, khả năng sáng tạo của mỗi cá nhân; nâng cao dân trí, phát triển nguồn nhân lực, bồi dưỡng nhân tài, đáp ứng yêu cầu của sự nghiệp xây dựng, bảo vệ Tổ quốc và hội nhập quốc tế. ",
        "id": "82b52978"
    },
"""
response = chat_mistral(prompt)
print(response)


1. Luật Giáo Dục nhằm đạt đến mục tiêu giáo dục nào trong số các mục tiêu sau đây?
   a) Phát triển toàn diện con người Việt Nam
   b) Nâng cao dân trí, phát triển nguồn nhân lực
   c) Bồi dưỡng nhân tài, đáp ứng yêu cầu của sự nghiệp xây dựng, bảo vệ Tổ quốc và hội nhập quốc tế
   d) Phát huy tiềm năng, khả năng sáng tạo của mỗi cá nhân
   e) Có phẩm chất, năng lực và ý thức công dân; có lòng yêu nước, tinh thần dân tộc, trung thành với lý tưởng độc lập dân tộc và chủ nghĩa xã hội

2. Chương I của Luật Giáo Dục nói về mục tiêu giáo dục chung nào trong các mục tiêu sau đây?
   a) Phát triển toàn diện con người Việt Nam
   b) Nâng cao dân trí, phát triển nguồn nhân lực
   c) Bồi dưỡng nhân tài, đáp ứng yêu cầu của sự nghiệp xây dựng, bảo vệ Tổ quốc và hội nhập quốc tế
   d) Phát huy tiềm năng, khả năng sáng tạo của mỗi cá nhân
   e) Có phẩm chất, năng lực và ý thức công dân; có lòng yêu nước, tinh thần dân tộc, trung thành với lý tưởng độc lập dân tộc và chủ nghĩa xã hội

3. Điều 2 của 

In [4]:
import os
from mistralai import Mistral

# Replace with your actual API key
CODESTRAL_MAMBA_API_KEY = CODESTRAL_MAMBA_API_KEY

client = Mistral(api_key=CODESTRAL_MAMBA_API_KEY)

# List available models
models = client.models.list()
print("Available models:", models)


Available models: object='list' data=[BaseModelCard(id='ministral-3b-2410', capabilities=ModelCapabilities(completion_chat=True, completion_fim=False, function_calling=True, fine_tuning=True, vision=False), object='model', created=1735796383, owned_by='mistralai', name='ministral-3b-2410', description='Official ministral-3b-2410 Mistral AI model', max_context_length=131072, aliases=['ministral-3b-latest'], deprecation=None, default_model_temperature=0.3, TYPE='base'), BaseModelCard(id='ministral-3b-latest', capabilities=ModelCapabilities(completion_chat=True, completion_fim=False, function_calling=True, fine_tuning=True, vision=False), object='model', created=1735796383, owned_by='mistralai', name='ministral-3b-2410', description='Official ministral-3b-2410 Mistral AI model', max_context_length=131072, aliases=['ministral-3b-2410'], deprecation=None, default_model_temperature=0.3, TYPE='base'), BaseModelCard(id='ministral-8b-2410', capabilities=ModelCapabilities(completion_chat=True, c